<h4 style="text-align: center; color: #BD6C37;"> <i> Ecole Polytechnique de Thiès <br>  Département Génie Informatique et Télécommunications </i> </h4>
<h1 style="text-align: center"> Principes MLOps </h1>
<h5 style="text-align: center">DIC3-GIT, 2023-2024</h5>
<h5 style="text-align: center">Mme Mously DIAW</h5>
<h1 style="text-align: center; color:#90edaa">Projet matière : Natural Language Processing with Disaster Tweets</h1>
<h5 style="text-align: center"> Par Kikia DIA, Mouhamadou Naby DIA, Ndeye Awa SALANE </h5>
<h3 style="text-align: center; color:#9000aa; text-decoration:underline"> II. Models Exploration </h3>


<a id="0"></a> <br>
### Sommaire
#### [Introduction](#1)
1. [Features Selection](#3)
1. [Vectorisation](#2)
1. [Models building](#4)
1. [Pipeline](#8)
#### [Conclusion](#5)
* <i>[References](#6)</i>
* <i>[Authors](#7)</i>

<a id="1"></a> 
#### Introduction [⏮️]()[👆🏽](#0)[⏭️](#2)

#### Import des librairies

In [6]:
# Ajouter le répertoire parent pour les imports de module
import sys
sys.path.append('..')

from gensim.models import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from nltk.tokenize import word_tokenize
import pandas as pd
#import pycaret
#from pycaret.classification import *
from pyngrok import ngrok
import ppscore as pps
import seaborn as sns
from settings.params import MODEL_PARAMS, SEED
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from src.data.make_dataset import get_dataset
from xgboost import XGBClassifier
import warnings

In [7]:
train, test = get_dataset(raw=False)

In [8]:
train.shape

(7591, 5)

In [9]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,deed reason earthquake may forgive,1
1,4,NaN,NaN,forest fire near la canada,1
2,5,NaN,NaN,resident ask shelter place notify officer evac...,1
3,6,NaN,NaN,receive wildfire evacuation order,1
4,7,NaN,NaN,get send photo ruby smoke pour school,1


In [10]:
# train = train.dropna(subset=['text'])

In [11]:
train.shape

(7591, 5)

<a id="2"></a> 
#### 1. Feature Selection [⏮️](#1)[👆🏽](#0)[⏭️](#3)

In [12]:
# Concatenate 'text' and 'Keywords' if 'Keywords' is not NaN
# train['combined_text'] = train.apply(
#     # lambda row: f"{row['text']} {row['keyword'] or ''} ".strip(),
#     lambda row: f"{row['text']} {row['keyword'] or ''} {row['location'] or ''}".strip(),

#     axis=1
# )
# X = train['combined_text']
X = train["text"]
y = train['target']
X

0                      deed reason earthquake may forgive
1                              forest fire near la canada
2       resident ask shelter place notify officer evac...
3                       receive wildfire evacuation order
4                   get send photo ruby smoke pour school
                              ...                        
7586     two giant crane hold bridge collapse nearby home
7587    aria control wild fire even northern part stat...
7588                                              volcano
7589    police investigate bike collide car little bik...
7590                     late home raze northern wildfire
Name: text, Length: 7591, dtype: object

##### CHI2 test

In [13]:


# # Sample data - Replace this with your tweets and labels
# data = train
# tweets = data["text"]
# labels = data["target"]

# # Convert text data to numerical features
# vectorizer = CountVectorizer(stop_words='english')
# X = vectorizer.fit_transform(tweets)


# # Apply Chi-Square test
# chi2_scores, p_values = chi2(X, y)

# # Create a DataFrame to view feature names and their scores
# feature_names = vectorizer.get_feature_names_out()
# chi2_df = pd.DataFrame({'Feature': feature_names, 'Chi2 Score': chi2_scores})

# # Sort by Chi2 Score in descending order
# chi2_df = chi2_df.sort_values(by='Chi2 Score', ascending=False)

# # Display the top features
# print(chi2_df.head(20))


<a id="2"></a> 
#### 2. Vectorisation [⏮️](#2)[👆🏽](#0)[⏭️](#4)

##### Bag of words

In [14]:
cv = CountVectorizer()

In [15]:
X_vect = cv.fit_transform(X)
X_vect = X_vect.toarray()

##### TF-IDF

In [17]:
tfidf = TfidfVectorizer()

In [18]:
X_vect = tfidf.fit_transform(X)
X_vect = X_vect
X_vect

<7591x6433 sparse matrix of type '<class 'numpy.float64'>'
	with 49247 stored elements in Compressed Sparse Row format>

In [19]:
df = pd.DataFrame(X_vect[2].T.todense(),
    	index=tfidf.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
df

,TF-IDF
shelter,0.584149
place,0.457637
notify,0.331479
resident,0.281756
ask,0.236822
...,...
festival,0.000000
ferry,0.000000
ferro,0.000000
fence,0.000000


#### Word2Vec

In [14]:
# # Preprocess the text data
# def preprocess_text(text):
#     # Tokenize the text
#     tokens = word_tokenize(text.lower())
#     return tokens

# # Apply preprocessing to the text data
# X_tokenized = X.apply(preprocess_text)

# # Train Word2Vec model
# word2vec_model = Word2Vec(sentences=X_tokenized, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# # Transform text data into vectors by averaging the word vectors
# def transform_text_to_vector(tokens, model):
#     word_vectors = [model.wv[word] for word in tokens if word in model.wv]
#     if len(word_vectors) == 0:
#         return np.zeros(model.vector_size)
#     return np.mean(word_vectors, axis=0)

# X_vect = X_tokenized.apply(lambda tokens: transform_text_to_vector(tokens, word2vec_model))
# X_vect = np.vstack(X_vect)



<a id="3"></a> 
#### 3. Model building [⏮️](#3)[👆🏽](#0)[⏭️](#5)

##### Train / Validation split


In [57]:
X_train_vect, X_val_vect, y_train, y_val = train_test_split(X_vect, y, test_size=0.2, random_state=50)

In [58]:
warnings.filterwarnings("ignore")

# Set up MLflow tracking
local_registry = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(local_registry)

# Create an experiment
exp_name = "disaster_tweets_classification"
experiment = mlflow.get_experiment_by_name(exp_name)
if not experiment:
    experiment_id = mlflow.create_experiment(exp_name)
else:
    experiment_id = experiment.experiment_id

#### Pycaret

In [59]:
# # Combine features and labels into a DataFrame
# df = pd.DataFrame(X_vect)
# df['target'] = y

# # Use PyCaret for classification
# clf = setup(data=df, target='target', session_id=123)

# # Compare different models
# best_model = compare_models()

# # Pull the results
# results = pull()

# # Print the results
# print(results)

# # Finalize the best model (optional)
# final_model = finalize_model(best_model)

# # Print the best model details
# print(final_model)

##### Grid Search for some models

In [60]:
import dagshub
dagshub.init(repo_owner='dnaby', repo_name='NLP-Disaster-Tweets-Detection', mlflow=True)

import mlflow


def train_and_evaluate_with_grid_search(model, param_grid, X_train, y_train, X_val, y_val, model_name):
    # Perform Grid Search
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                               cv=5, n_jobs=-1, verbose=2, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    
    # Get the best model from Grid Search
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    
    with mlflow.start_run(run_name=model_name, experiment_id=experiment_id):
        # Train the best model
        best_model.fit(X_train, y_train)

        # Make predictions
        y_train_pred = best_model.predict(X_train)
        y_val_pred = best_model.predict(X_val)

        # Calculate metrics for training set
        train_f1 = f1_score(y_train, y_train_pred, average='weighted')
        train_precision = precision_score(y_train, y_train_pred, average='weighted')
        train_recall = recall_score(y_train, y_train_pred, average='weighted')

        # Calculate metrics for validation set
        val_f1 = f1_score(y_val, y_val_pred, average='weighted')
        val_precision = precision_score(y_val, y_val_pred, average='weighted')
        val_recall = recall_score(y_val, y_val_pred, average='weighted')

        # Log parameters and metrics
        mlflow.log_params(best_params)
        mlflow.log_metric("train_f1_score", train_f1)
        mlflow.log_metric("train_precision", train_precision)
        mlflow.log_metric("train_recall", train_recall)
        mlflow.log_metric("val_f1_score", val_f1)
        mlflow.log_metric("val_precision", val_precision)
        mlflow.log_metric("val_recall", val_recall)

        # Log the best model
        #mlflow.sklearn.log_model(best_model, model_name, input_example=X_train[:30])

        # Print metrics
        print(f'{model_name} Training F1 Score: {train_f1}')
        print(f'{model_name} Training Precision: {train_precision}')
        print(f'{model_name} Training Recall: {train_recall}')
        print(f'{model_name} Validation F1 Score: {val_f1}')
        print(f'{model_name} Validation Precision: {val_precision}')
        print(f'{model_name} Validation Recall: {val_recall}')

        # Print best parameters
        print(f'Best parameters for {model_name}: {best_params}')
    return grid_search.best_estimator_



Initialized MLflow to track repo "dnaby/NLP-Disaster-Tweets-Detection"

Repository dnaby/NLP-Disaster-Tweets-Detection initialized!

In [61]:
# Define classification models with fixed random seed
models = {
    'RandomForestClassifier': RandomForestClassifier(random_state=SEED),
    'SVC': SVC(C=100, gamma='scale', kernel='linear', random_state=SEED),
    'Naive Bayes': MultinomialNB(),
    'KNN': KNeighborsClassifier(),
    'XGBoost': XGBClassifier(eval_metric='logloss', random_state=SEED),
    'Logistic Regression': LogisticRegression()
}

# Parameter grids for each model
param_grids = {
    'RandomForestClassifier': [{
        'n_estimators': [10, 50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }],
    'SVC': [{
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto'],
        'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
    }],
    'Naive Bayes': [{
        'alpha': [0.01, 0.1, 1, 10]
    }],
    'KNN': [{
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan', 'minkowski']
    }],
    'XGBoost': [{
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 6, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.6, 0.8, 1.0]
    }],
    'Logistic Regression': [{
        'C': [0.01, 0.1, 0.8, 1, 10, 100],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        'max_iter': [100, 200, 500, 1000],
        'penalty': ['l1', 'l2', 'elasticnet', 'none'],
        'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]  # Only used if penalty is 'elasticnet'
    }]
}


In [62]:
# Perform Grid Search on each model
best_models = {}
for model_name, model in models.items():
    print(f"Performing Grid Search for {model_name}...")
    best_models[model_name] = train_and_evaluate_with_grid_search(model, param_grids[model_name], X_train_vect, y_train, X_val_vect, y_val, model_name)

Performing Grid Search for RandomForestClassifier...
Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   1.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   1.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   1.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   1.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   1.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.9s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   5.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   5.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_

/opt/homebrew/Caskroom/miniforge/base/envs/final_project/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/final_project/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/final_project/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/final_project/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/final_project/lib

[CV] END C=0.01, l1_ratio=0.1, max_iter=200, penalty=l1, solver=sag; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=200, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=200, penalty=l1, solver=sag; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=200, penalty=l1, solver=sag; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=200, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=200, penalty=l1, solver=sag; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=200, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=100, penalty=elasticnet, solver=saga; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=200, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=100, penalty=elasticnet, solver=saga; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=200, penalty=l1, solver=liblinear; tot

/opt/homebrew/Caskroom/miniforge/base/envs/final_project/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/final_project/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/final_project/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/final_project/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/final_project/lib

[CV] END C=0.01, l1_ratio=0.1, max_iter=1000, penalty=elasticnet, solver=newton-cg; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.3, max_iter=100, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=1000, penalty=elasticnet, solver=newton-cg; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=1000, penalty=elasticnet, solver=newton-cg; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=1000, penalty=elasticnet, solver=lbfgs; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=1000, penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=1000, penalty=elasticnet, solver=lbfgs; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.3, max_iter=100, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=1000, penalty=elasticnet, solver=lbfgs; total time=   0.0s
[CV] END C=0.01, l1_ratio=0.1, max_iter=1000, penalty=elasticnet, solver=lbfgs; total time=   0.0s
[CV] END C=0

...

[CV] END C=100, l1_ratio=0.9, max_iter=1000, penalty=elasticnet, solver=saga; total time= 2.3min


/opt/homebrew/Caskroom/miniforge/base/envs/final_project/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END C=100, l1_ratio=0.9, max_iter=1000, penalty=elasticnet, solver=saga; total time= 2.2min


/opt/homebrew/Caskroom/miniforge/base/envs/final_project/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END C=100, l1_ratio=0.9, max_iter=1000, penalty=elasticnet, solver=saga; total time= 2.2min
Logistic Regression Training F1 Score: 0.865930319145606
Logistic Regression Training Precision: 0.8735002484630336
Logistic Regression Training Recall: 0.8680830039525692
Logistic Regression Validation F1 Score: 0.7828717917840872
Logistic Regression Validation Precision: 0.7952339489457985
Logistic Regression Validation Recall: 0.7880184331797235
Best parameters for Logistic Regression: {'C': 1, 'l1_ratio': 0.1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'newton-cg'}


In [65]:
best_models

{'RandomForestClassifier': RandomForestClassifier(min_samples_split=5, n_estimators=200, random_state=42),
 'SVC': SVC(C=1, random_state=42),
 'Naive Bayes': MultinomialNB(alpha=1),
 'KNN': KNeighborsClassifier(metric='euclidean', n_neighbors=9, weights='distance'),
 'XGBoost': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric='logloss',
               feature_types=None, gamma=None, grow_policy=None,
               importance_type=None, interaction_constraints=None,
               learning_rate=0.1, max_bin=None, max_cat_threshold=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, multi_strategy=None, n_estimators=200,
               n_jobs=None, 

In [74]:
# Define a function to train and evaluate models
def train_and_evaluate(model, X_train, y_train, X_val, y_val, model_name):
    with mlflow.start_run(run_name=model_name, experiment_id=experiment_id) as run:
        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        # Calculate metrics for training set
        train_f1 = f1_score(y_train, y_train_pred, average='weighted')
        train_precision = precision_score(y_train, y_train_pred, average='weighted')
        train_recall = recall_score(y_train, y_train_pred, average='weighted')

        # Calculate metrics for validation set
        val_f1 = f1_score(y_val, y_val_pred, average='weighted')
        val_precision = precision_score(y_val, y_val_pred, average='weighted')
        val_recall = recall_score(y_val, y_val_pred, average='weighted')

        # Log parameters and metrics
        mlflow.log_params(model.get_params())
        mlflow.log_metric("train_f1_score", train_f1)
        mlflow.log_metric("train_precision", train_precision)
        mlflow.log_metric("train_recall", train_recall)
        mlflow.log_metric("val_f1_score", val_f1)
        mlflow.log_metric("val_precision", val_precision)
        mlflow.log_metric("val_recall", val_recall)

        # Log the model
        #mlflow.sklearn.log_model(model, model_name)

        # Print metrics
        print(f'{model_name} Training F1 Score: {train_f1}')
        print(f'{model_name} Training Precision: {train_precision}')
        print(f'{model_name} Training Recall: {train_recall}')
        print(f'{model_name} Validation F1 Score: {val_f1}')
        print(f'{model_name} Validation Precision: {val_precision}')
        print(f'{model_name} Validation Recall: {val_recall}')

#### Single model evaluation

In [69]:
# train_and_evaluate(model, param_grids[model_name], X_train_vect, y_train, X_val_vect, y_val, model_name)

In [75]:
for model_name, model in models.items():
    train_and_evaluate(model,  X_train_vect, y_train, X_val_vect, y_val, model_name)

RandomForestClassifier Training F1 Score: 0.9825129243531694
RandomForestClassifier Training Precision: 0.9827214589736575
RandomForestClassifier Training Recall: 0.9825428194993412
RandomForestClassifier Validation F1 Score: 0.785077429502107
RandomForestClassifier Validation Precision: 0.794333533088317
RandomForestClassifier Validation Recall: 0.7893350888742594
SVC Training F1 Score: 0.9783892146876446
SVC Training Precision: 0.9785830852112306
SVC Training Recall: 0.9784255599472991
SVC Validation F1 Score: 0.7344832577950979
SVC Validation Precision: 0.7343245313392532
SVC Validation Recall: 0.7346938775510204
Naive Bayes Training F1 Score: 0.8726595351822098
Naive Bayes Training Precision: 0.878209912820674
Naive Bayes Training Recall: 0.8743412384716732
Naive Bayes Validation F1 Score: 0.7844397627160873
Naive Bayes Validation Precision: 0.7912577934399565
Naive Bayes Validation Recall: 0.7880184331797235
KNN Training F1 Score: 0.825387430138518
KNN Training Precision: 0.831286

##### Voting & Stacking Classifiers with best hyperparameters

In [77]:
best_models = {
  'RandomForestClassifier': RandomForestClassifier(min_samples_split=5, n_estimators=200, random_state=42),
  'SVC': SVC(C=1, random_state=42),
  'Naive Bayes': MultinomialNB(alpha=1),
  'KNN': KNeighborsClassifier(metric='euclidean', n_neighbors=9, weights='distance'),
  'XGBoost': XGBClassifier(
    enable_categorical=False, 
    eval_metric='logloss',
    learning_rate=0.1, 
    max_depth=10, 
    n_estimators=200, 
    random_state=42
  ),
  'Logistic Regression': LogisticRegression(C=1, l1_ratio=0.1, solver='newton-cg')
}

In [85]:
# Create a VotingClassifier with the best models
voting_clf = VotingClassifier(estimators=[
    #('rf', best_models['RandomForestClassifier']),
    #('svc', best_models['SVC']),
    ('nb', best_models['Naive Bayes']),
    #('knn', best_models['KNN']),
    ('xgb', best_models['XGBoost']),
    ('lr', best_models['Logistic Regression']),
], voting='hard')

stacking_clf = StackingClassifier(estimators=[
    #('rf', best_models['RandomForestClassifier']),
    #('svc', best_models['SVC']),
    ('nb', best_models['Naive Bayes']),
    #('knn', best_models['KNN']),
    ('xgb', best_models['XGBoost']),
    ('lr', best_models['Logistic Regression']),
], final_estimator=LogisticRegression(),
    cv=5)

In [86]:
def train_and_evaluate_ensemble(model, X_train, y_train, X_val, y_val, model_name):
    with mlflow.start_run(run_name=model_name, experiment_id=experiment_id) as run:
        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        # Calculate metrics for training set
        train_f1 = f1_score(y_train, y_train_pred, average='weighted')
        train_precision = precision_score(y_train, y_train_pred, average='weighted')
        train_recall = recall_score(y_train, y_train_pred, average='weighted')

        # Calculate metrics for validation set
        val_f1 = f1_score(y_val, y_val_pred, average='weighted')
        val_precision = precision_score(y_val, y_val_pred, average='weighted')
        val_recall = recall_score(y_val, y_val_pred, average='weighted')

        # Log parameters and metrics
        mlflow.log_param("ensemble_method", "VotingClassifier")
        mlflow.log_metric("train_f1_score", train_f1)
        mlflow.log_metric("train_precision", train_precision)
        mlflow.log_metric("train_recall", train_recall)
        mlflow.log_metric("val_f1_score", val_f1)
        mlflow.log_metric("val_precision", val_precision)
        mlflow.log_metric("val_recall", val_recall)

        # Log the model
        #mlflow.sklearn.log_model(model, model_name, input_example=X_train[:30])

        # Print metrics
        print(f'{model_name} Training F1 Score: {train_f1}')
        print(f'{model_name} Training Precision: {train_precision}')
        print(f'{model_name} Training Recall: {train_recall}')
        print(f'{model_name} Validation F1 Score: {val_f1}')
        print(f'{model_name} Validation Precision: {val_precision}')
        print(f'{model_name} Validation Recall: {val_recall}')

# Train and evaluate the ensemble model
train_and_evaluate_ensemble(voting_clf, X_train_vect, y_train, X_val_vect, y_val, "Ensemble_VotingClassifier")


Ensemble_VotingClassifier Training F1 Score: 0.8731924275716889
Ensemble_VotingClassifier Training Precision: 0.8816286587224107
Ensemble_VotingClassifier Training Recall: 0.8753293807641633
Ensemble_VotingClassifier Validation F1 Score: 0.789901358200096
Ensemble_VotingClassifier Validation Precision: 0.8014607015133034
Ensemble_VotingClassifier Validation Recall: 0.7946017116524029


In [87]:
train_and_evaluate_ensemble(stacking_clf, X_train_vect, y_train, X_val_vect, y_val, "Ensemble_StackingClassifier")

Ensemble_StackingClassifier Training F1 Score: 0.8827947873394854
Ensemble_StackingClassifier Training Precision: 0.8868364402783078
Ensemble_StackingClassifier Training Recall: 0.8840579710144928
Ensemble_StackingClassifier Validation F1 Score: 0.7950635327545075
Ensemble_StackingClassifier Validation Precision: 0.800236914114714
Ensemble_StackingClassifier Validation Recall: 0.7978933508887426


In [ ]:
# Terminate open tunnels if they exist
ngrok.kill()
NGROK_AUTH_TOKEN = "2iXzpfikPynAXlcC1qn3NDhkhMS_7BbPuGQ5Sc8R7aohgotz6"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(5000)
print("MLflow Tracking UI:", public_url)

# Transition models to production
client = MlflowClient()
for model_name in models.keys():
    try:
        model_versions = client.search_model_versions(f"name='{model_name}'")
        if model_versions:
            # Get the latest model version
            latest_version = model_versions[-1]
            client.transition_model_version_stage(
                name=model_name,
                version=latest_version.version,
                stage="production"
            )
    except Exception as e:
        print(f"Error transitioning model {model_name}: {e}")


MLflow Tracking UI: NgrokTunnel: "https://5c31-41-82-155-235.ngrok-free.app" -> "http://localhost:5000"



<a id="3"></a> 
#### 4. Pipeline [⏮️](#4)[👆🏽](#0)[⏭️](#6)

In [ ]:
idf_pip=Pipeline(
                [
                    ('tf_idf',TfidfVectorizer(ngram_range=(1,1))),
                    ('model',LogisticRegression(C=.8,solver='sag',max_iter=1000))
                ]
                )
idf_pip.fit(X_train,y_train)

<a id="5"></a> 
#### Conclusion [⏮️](#4)[👆🏽](#0)[⏭️](#6)


<a id="6"></a> 
#### <i>References</i> [⏮️](#5)[👆🏽](#0)[⏭️](#7)
Here is some text with a reference to the [Python documentation](https://docs.python.org/).

...

Here are some references for more information on the libraries used:

- [Pandas documentation](https://pandas.pydata.org/docs/)
- [NumPy documentation](https://numpy.org/doc/stable/)
- [Tuning the hyper-parameters of an estimator](https://scikit-learn.org/stable/modules/grid_search.html)

🍀 Auteurs
- 🧑🏾‍💻 Kikia DIA
- 🧑🏾‍💻 Mouhamadou Naby DIA
- 🧑🏾‍💻 Ndeye Awa SALANE

🍀 Affiliations
- 🎓 Ecole Polytechnique de THIES

🍀 Département 
- 💻 Génie Informatique et Télécommunications

🍀 Niveau
- 📚 DIC3